In [47]:
import doctest
import functools
import operator


In [2]:
test_input_1 = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""


In [65]:
def parse_round(r: str):
  """
  >>> sorted(parse_round('3 blue, 4 red').items())
  [('blue', 3), ('red', 4)]
  >>> sorted(parse_round('1 x, 2 y, 10 z').items())
  [('x', 1), ('y', 2), ('z', 10)]
  """
  cubes = [c.split(' ') for c in r.split(', ')]
  return {color: int(n) for (n, color) in cubes}

def parse_game(g: str):
  """
  >>> num, rounds = parse_game('Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue')
  >>> num
  2
  >>> [sorted(r.items()) for r in rounds]
  [[('blue', 1), ('green', 2)], [('blue', 4), ('green', 3), ('red', 1)], [('blue', 1), ('green', 1)]]
  """
  game, rounds = g.split(': ')
  rounds = [parse_round(r) for r in rounds.split('; ')]
  return int(game[5:]), rounds

def parse_games(s: str):
  return dict(parse_game(l) for l in s.splitlines())

def is_possible_round(round, bag):
  r"""
  >>> is_possible_round({'x': 5}, {'x': 4})
  False
  >>> is_possible_round({'x': 5, 'y': 1}, {'x': 7, 'y': 2})
  True
  >>> is_possible_round({'x': 5, 'y': 5}, {'x': 7, 'y': 2})
  False
  """
  return all(bag.get(color, 0) >= count for color, count in round.items())

def is_possible_game(rounds, bag):
  r"""
  >>> is_possible_game([{'x': 5}, {'x': 5, 'y': 1}, {'y': 1}], {'x': 5, 'y': 1})
  True
  >>> is_possible_game([{'x': 1}, {'x': 3, 'y': 3}], {'x': 5, 'y': 1})
  False
  """
  return all(is_possible_round(r, bag) for r in rounds)

def possible_games(gs, bag):
  for num, rounds in gs.items():
    if is_possible_game(rounds, bag):
      yield num, rounds

def solution_1(gs):
  bag = {'red': 12, 'green': 13, 'blue': 14}
  return sum(num for (num, _) in possible_games(gs, bag))

# Part 2.

def acc_max_value(d1, d2):
  return {k: max(d1.get(k, 0), d2.get(k, 0)) for k in d1.keys() | d2.keys()}

def min_cubes(rounds):
  """
  >>> _, g = parse_game("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green")
  >>> sorted(min_cubes(g).items())
  [('blue', 6), ('green', 2), ('red', 4)]
  >>> _, g = parse_game("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red")
  >>> sorted(min_cubes(g).items())
  [('blue', 6), ('green', 13), ('red', 20)]
  """
  return functools.reduce(acc_max_value, rounds)

def power(cubes):
  """
  >>> power({'blue': 6, 'green': 13, 'red': 20})
  1560
  """
  return functools.reduce(operator.mul, cubes.values(), 1)

def solution_2(games):
  return sum(power(min_cubes(g)) for g in games.values())


In [66]:
doctest.testmod(verbose=True)


Trying:
    is_possible_game([{'x': 5}, {'x': 5, 'y': 1}, {'y': 1}], {'x': 5, 'y': 1})
Expecting:
    True
ok
Trying:
    is_possible_game([{'x': 1}, {'x': 3, 'y': 3}], {'x': 5, 'y': 1})
Expecting:
    False
ok
Trying:
    is_possible_round({'x': 5}, {'x': 4})
Expecting:
    False
ok
Trying:
    is_possible_round({'x': 5, 'y': 1}, {'x': 7, 'y': 2})
Expecting:
    True
ok
Trying:
    is_possible_round({'x': 5, 'y': 5}, {'x': 7, 'y': 2})
Expecting:
    False
ok
Trying:
    _, g = parse_game("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green")
Expecting nothing
ok
Trying:
    sorted(min_cubes(g).items())
Expecting:
    [('blue', 6), ('green', 2), ('red', 4)]
ok
Trying:
    _, g = parse_game("Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red")
Expecting nothing
ok
Trying:
    sorted(min_cubes(g).items())
Expecting:
    [('blue', 6), ('green', 13), ('red', 20)]
ok
Trying:
    num, rounds = parse_game('Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 bl

TestResults(failed=0, attempted=15)

In [63]:
test_games = parse_games(test_input_1)
print(solution_1(test_games))
print(solution_2(test_games))


8
2286


In [64]:
# Final answers
with open('../data/day02.txt') as f:
    games = parse_games(f.read().strip())
    print('Part 1: ', solution_1(games))
    print('Part 2: ', solution_2(games))


Part 1:  1734
Part 2:  70387
